# Практика: Введение в NLP

В этом ноутбуке мы пройдём основные концепции NLP на практике:
- Токенизация
- Морфологический анализ
- Построение пайплайна
- Классификация текстов

Убедитесь, что установлены необходимые библиотеки:
```
pip install nltk pymystem3 pymorphy2 scikit-learn
```

## 1. Токенизация и сегментация

In [1]:
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
    nltk.download('stopwords')

# Примеры текстов
text_ru = """Привет, мир! Это первый пример. NLP очень интересен.
Машинное обучение меняет нашу жизнь."""

text_en = """Hello, world! This is the first example. NLP is very interesting.
Machine learning is changing our lives."""

print("=" * 60)
print("СЕГМЕНТАЦИЯ НА ПРЕДЛОЖЕНИЯ")
print("=" * 60)
print("\nРусский текст:")
sentences_ru = sent_tokenize(text_ru, language='russian')
for i, sent in enumerate(sentences_ru, 1):
    print(f"{i}. {sent}")

print("\nАнглийский текст:")
sentences_en = sent_tokenize(text_en, language='english')
for i, sent in enumerate(sentences_en, 1):
    print(f"{i}. {sent}")

СЕГМЕНТАЦИЯ НА ПРЕДЛОЖЕНИЯ

Русский текст:
1. Привет, мир!
2. Это первый пример.
3. NLP очень интересен.
4. Машинное обучение меняет нашу жизнь.

Английский текст:
1. Hello, world!
2. This is the first example.
3. NLP is very interesting.
4. Machine learning is changing our lives.


In [2]:
print("=" * 60)
print("ТОКЕНИЗАЦИЯ")
print("=" * 60)

# Простая токенизация по пробелам
simple_tokens = text_ru.split()
print(f"\nПростая токенизация (split):")
print(f"Токены: {simple_tokens}")
print(f"Количество: {len(simple_tokens)}")
print("⚠️ Проблема: пунктуация слилась со словами")

# Более правильная токенизация
print("\nТокенизация с разделением пунктуации:")
tokens_ru = word_tokenize(text_ru, language='russian')
print(f"Токены: {tokens_ru[:20]}...")  # Первые 20
print(f"Количество: {len(tokens_ru)}")

print("\nАнглийская токенизация:")
tokens_en = word_tokenize(text_en)
print(f"Токены: {tokens_en[:20]}...")
print(f"Количество: {len(tokens_en)}")

ТОКЕНИЗАЦИЯ

Простая токенизация (split):
Токены: ['Привет,', 'мир!', 'Это', 'первый', 'пример.', 'NLP', 'очень', 'интересен.', 'Машинное', 'обучение', 'меняет', 'нашу', 'жизнь.']
Количество: 13
⚠️ Проблема: пунктуация слилась со словами

Токенизация с разделением пунктуации:
Токены: ['Привет', ',', 'мир', '!', 'Это', 'первый', 'пример', '.', 'NLP', 'очень', 'интересен', '.', 'Машинное', 'обучение', 'меняет', 'нашу', 'жизнь', '.']...
Количество: 18

Английская токенизация:
Токены: ['Hello', ',', 'world', '!', 'This', 'is', 'the', 'first', 'example', '.', 'NLP', 'is', 'very', 'interesting', '.', 'Machine', 'learning', 'is', 'changing', 'our']...
Количество: 22


In [3]:
# Пользовательская токенизация с regex
print("\nОсовещение токенизации с regex:")

def custom_tokenize(text):
    # Разделяем по пробелам, но сохраняем пунктуацию
    # Паттерн: слово или одиночный символ
    pattern = r"\b\w+\b|[.,!?;:-]"
    tokens = re.findall(pattern, text, re.UNICODE)
    return tokens

custom_tokens = custom_tokenize(text_ru.lower())
print(f"Custom токены: {custom_tokens}")


Осовещение токенизации с regex:
Custom токены: ['привет', ',', 'мир', '!', 'это', 'первый', 'пример', '.', 'nlp', 'очень', 'интересен', '.', 'машинное', 'обучение', 'меняет', 'нашу', 'жизнь', '.']


## 2. Морфологический анализ

In [4]:
# Для русского языка используем pymorphy2
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

# Примеры слов
words = ['бежали', 'конца', 'банка', 'красивая', 'работают']

print("=" * 60)
print("ЛЕММАТИЗАЦИЯ И МОРФОЛОГИЧЕСКИЙ АНАЛИЗ")
print("=" * 60)

for word in words:
    p = morph.parse(word)[0]
    print(f"\nСлово: '{word}'")
    print(f"  Лемма: {p.normal_form}")
    print(f"  Часть речи: {p.tag.POS}")
    print(f"  Грамм. признаки: {p.tag}")

ЛЕММАТИЗАЦИЯ И МОРФОЛОГИЧЕСКИЙ АНАЛИЗ

Слово: 'бежали'
  Лемма: бежать
  Часть речи: VERB
  Грамм. признаки: VERB,perf,intr plur,past,indc

Слово: 'конца'
  Лемма: конец
  Часть речи: NOUN
  Грамм. признаки: NOUN,inan,masc sing,gent

Слово: 'банка'
  Лемма: банк
  Часть речи: NOUN
  Грамм. признаки: NOUN,inan,masc sing,gent

Слово: 'красивая'
  Лемма: красивый
  Часть речи: ADJF
  Грамм. признаки: ADJF,Qual femn,sing,nomn

Слово: 'работают'
  Лемма: работать
  Часть речи: VERB
  Грамм. признаки: VERB,impf,intr plur,3per,pres,indc


In [5]:
# Лемматизация полного текста
print("\n" + "=" * 60)
print("ЛЕММАТИЗАЦИЯ ТЕКСТА")
print("=" * 60)

def lemmatize_text(text):
    # Токенизируем
    tokens = word_tokenize(text.lower(), language='russian')
    # Лемматизируем
    lemmas = []
    for token in tokens:
        if token.isalpha():  # Только буквы
            parsed = morph.parse(token)[0]
            lemmas.append(parsed.normal_form)
        else:
            lemmas.append(token)  # Числа, пунктуация как есть
    return lemmas

original = "Мама мыла раму. Дети бежали на площадку."
lemmatized = lemmatize_text(original)

print(f"\nОригинал: {original}")
print(f"Леммы: {' '.join(lemmatized)}")


ЛЕММАТИЗАЦИЯ ТЕКСТА

Оригинал: Мама мыла раму. Дети бежали на площадку.
Леммы: мама мыло рама . ребёнок бежать на площадка .


In [6]:
# Анализ одной словоформы с несколькими вариантами
print("\n" + "=" * 60)
print("ОМОНИМИЯ: СЛОВО С НЕСКОЛЬКИМИ ЗНАЧЕНИЯМИ")
print("=" * 60)

ambiguous_word = 'банка'
parses = morph.parse(ambiguous_word)

print(f"\nВсе возможные разборы слова '{ambiguous_word}':")
for i, p in enumerate(parses, 1):
    print(f"{i}. Лемма: {p.normal_form:15} | Часть речи: {p.tag.POS:10} | Полный тег: {p.tag}")


ОМОНИМИЯ: СЛОВО С НЕСКОЛЬКИМИ ЗНАЧЕНИЯМИ

Все возможные разборы слова 'банка':
1. Лемма: банк            | Часть речи: NOUN       | Полный тег: NOUN,inan,masc sing,gent
2. Лемма: банка           | Часть речи: NOUN       | Полный тег: NOUN,inan,femn sing,nomn


## 3. Построение простого пайплайна

In [7]:
print("=" * 60)
print("ПОЛНЫЙ ПАЙПЛАЙН ПРЕДОБРАБОТКИ")
print("=" * 60)

class TextPipeline:
    def __init__(self, language='russian'):
        self.language = language
        self.morph = pymorphy2.MorphAnalyzer()
        if language == 'russian':
            self.stop_words = stopwords.words('russian')
        else:
            self.stop_words = stopwords.words('english')
    
    def process(self, text):
        """Полная обработка текста"""
        # 1. Сегментация на предложения
        sentences = sent_tokenize(text, language=self.language)
        
        result = {
            'original': text,
            'sentences': sentences,
            'tokens': [],
            'lemmas': [],
            'cleaned_tokens': []
        }
        
        # 2. Токенизация и лемматизация
        all_tokens = word_tokenize(text.lower(), language=self.language)
        result['tokens'] = all_tokens
        
        # 3. Лемматизация
        lemmas = []
        for token in all_tokens:
            if token.isalpha():
                lemma = self.morph.parse(token)[0].normal_form
                lemmas.append(lemma)
            else:
                lemmas.append(token)
        result['lemmas'] = lemmas
        
        # 4. Удаление стоп-слов и пунктуации
        cleaned = [w for w in lemmas if w.isalpha() and w not in self.stop_words]
        result['cleaned_tokens'] = cleaned
        
        return result

# Использование
pipeline = TextPipeline('russian')

test_text = """Машинное обучение и обработка естественного языка — это мощные инструменты.
Они помогают нам понимать и анализировать текст."""

result = pipeline.process(test_text)

print(f"\nОригинальный текст:\n{result['original']}")
print(f"\nПредложений: {len(result['sentences'])}")
for i, sent in enumerate(result['sentences'], 1):
    print(f"  {i}. {sent}")

print(f"\nВсего токенов: {len(result['tokens'])}")
print(f"Первые 10 токенов: {result['tokens'][:10]}")

print(f"\nОчищенные токены (без стоп-слов): {len(result['cleaned_tokens'])}")
print(f"Токены: {result['cleaned_tokens']}")

ПОЛНЫЙ ПАЙПЛАЙН ПРЕДОБРАБОТКИ

Оригинальный текст:
Машинное обучение и обработка естественного языка — это мощные инструменты.
Они помогают нам понимать и анализировать текст.

Предложений: 2
  1. Машинное обучение и обработка естественного языка — это мощные инструменты.
  2. Они помогают нам понимать и анализировать текст.

Всего токенов: 19
Первые 10 токенов: ['машинное', 'обучение', 'и', 'обработка', 'естественного', 'языка', '—', 'это', 'мощные', 'инструменты']

Очищенные токены (без стоп-слов): 12
Токены: ['машинный', 'обучение', 'обработка', 'естественный', 'язык', 'это', 'мощный', 'инструмент', 'помогать', 'понимать', 'анализировать', 'текст']


## 4. Классификация текстов

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

print("=" * 60)
print("КЛАССИФИКАЦИЯ НОВОСТЕЙ НА РУССКОМ")
print("=" * 60)

# Синтетический корпус для демонстрации
# В реальности нужна большая размеченная выборка
train_texts = [
    "Президент подписал новый указ о реформе образования",
    "Правительство приняло бюджет на следующий год",
    "Депутаты голосовали за законопроект",
    "Российская сборная выиграла золотую медаль на олимпиаде",
    "Спортсмены готовятся к чемпионату мира",
    "Футболист забил решающий гол в матче",
    "Новая модель смартфона выходит на рынок",
    "Компания объявила о разработке искусственного интеллекта",
    "Ученые создали инновационную технологию",
]

train_labels = [
    0, 0, 0,  # политика
    1, 1, 1,  # спорт
    2, 2, 2,  # технология
]

# Векторизация
vectorizer = TfidfVectorizer(lowercase=True, analyzer='char', ngram_range=(2, 3))
X_train = vectorizer.fit_transform(train_texts)

print(f"\nМатрица TF-IDF формы: {X_train.shape}")
print(f"Число признаков: {len(vectorizer.get_feature_names_out())}")

# Обучение классификатора
clf = LogisticRegression(random_state=42, max_iter=200)
clf.fit(X_train, train_labels)

print("\nМодель обучена!")

КЛАССИФИКАЦИЯ НОВОСТЕЙ НА РУССКОМ

Матрица TF-IDF формы: (9, 514)
Число признаков: 514

Модель обучена!


In [9]:
# Тестирование
test_texts = [
    "Депутаты обсуждают новый закон",  # Политика
    "Спортсмен выиграл чемпионат",     # Спорт
    "Новая технология в смартфоне",    # Технология
]

X_test = vectorizer.transform(test_texts)
predictions = clf.predict(X_test)
probabilities = clf.predict_proba(X_test)

label_names = ['Политика', 'Спорт', 'Технология']

print("\nПредсказания для тестовых текстов:")
print("-" * 60)

for text, pred, proba in zip(test_texts, predictions, probabilities):
    print(f"\nТекст: {text}")
    print(f"Предсказание: {label_names[pred]} (уверенность: {proba[pred]:.2%})")
    print(f"Вероятности по классам:")
    for label, prob in zip(label_names, proba):
        bar = '█' * int(prob * 30)
        print(f"  {label:15} {prob:6.2%} {bar}")


Предсказания для тестовых текстов:
------------------------------------------------------------

Текст: Депутаты обсуждают новый закон
Предсказание: Политика (уверенность: 43.72%)
Вероятности по классам:
  Политика        43.72% █████████████
  Спорт           27.97% ████████
  Технология      28.31% ████████

Текст: Спортсмен выиграл чемпионат
Предсказание: Спорт (уверенность: 45.77%)
Вероятности по классам:
  Политика        26.66% ███████
  Спорт           45.77% █████████████
  Технология      27.57% ████████

Текст: Новая технология в смартфоне
Предсказание: Технология (уверенность: 44.06%)
Вероятности по классам:
  Политика        26.77% ████████
  Спорт           29.17% ████████
  Технология      44.06% █████████████


In [10]:
# На обучающей выборке
predictions_train = clf.predict(X_train)
print("\n" + "=" * 60)
print("МЕТРИКИ КАЧЕСТВА НА ОБУЧАЮЩЕЙ ВЫБОРКЕ")
print("=" * 60)

print(f"\nОбщая точность: {(predictions_train == train_labels).mean():.2%}")
print("\nОтчёт по классам:")
print(classification_report(train_labels, predictions_train, target_names=label_names))

print("\nМатрица ошибок:")
cm = confusion_matrix(train_labels, predictions_train)
print(cm)


МЕТРИКИ КАЧЕСТВА НА ОБУЧАЮЩЕЙ ВЫБОРКЕ

Общая точность: 100.00%

Отчёт по классам:
              precision    recall  f1-score   support

    Политика       1.00      1.00      1.00         3
       Спорт       1.00      1.00      1.00         3
  Технология       1.00      1.00      1.00         3

    accuracy                           1.00         9
   macro avg       1.00      1.00      1.00         9
weighted avg       1.00      1.00      1.00         9


Матрица ошибок:
[[3 0 0]
 [0 3 0]
 [0 0 3]]


## 5. Практические задачи

In [11]:
print("=" * 60)
print("ЗАДАЧА 1: Анализ сложности омонимии")
print("=" * 60)

# Найдите слова, которые имеют несколько разборов
ambiguous_words = ['коса', 'замок', 'мука', 'хлеб', 'роман']

print("\nАнализ омонимичных слов:")
for word in ambiguous_words:
    parses = morph.parse(word)
    print(f"\n'{word}': {len(parses)} вариант(ов)")
    for i, p in enumerate(parses[:3], 1):  # Первые 3
        print(f"  {i}. {p.normal_form} ({p.tag.POS})")

ЗАДАЧА 1: Анализ сложности омонимии

Анализ омонимичных слов:

'коса': 3 вариант(ов)
  1. коса (NOUN)
  2. кос (NOUN)
  3. косой (ADJS)

'замок': 3 вариант(ов)
  1. замок (NOUN)
  2. замок (NOUN)
  3. замокнуть (VERB)

'мука': 1 вариант(ов)
  1. мука (NOUN)

'хлеб': 2 вариант(ов)
  1. хлеб (NOUN)
  2. хлеб (NOUN)

'роман': 12 вариант(ов)
  1. роман (NOUN)
  2. роман (NOUN)
  3. роман (NOUN)


In [12]:
print("\n" + "=" * 60)
print("ЗАДАЧА 2: Сравнение токенизаций")
print("=" * 60)

complex_text = "Mr. Smith e-mailed the file at 5:00 p.m. to john@example.com"

# Попробуем разные подходы
simple = complex_text.split()
nltk_tokens = word_tokenize(complex_text)

print(f"\nТекст: {complex_text}")
print(f"\nПростая токенизация (split): {len(simple)} токенов")
print(simple)
print(f"\nNLTK токенизация: {len(nltk_tokens)} токенов")
print(nltk_tokens)
print("\nЗамечание: NLTK лучше разделяет пунктуацию")


ЗАДАЧА 2: Сравнение токенизаций

Текст: Mr. Smith e-mailed the file at 5:00 p.m. to john@example.com

Простая токенизация (split): 10 токенов
['Mr.', 'Smith', 'e-mailed', 'the', 'file', 'at', '5:00', 'p.m.', 'to', 'john@example.com']

NLTK токенизация: 12 токенов
['Mr.', 'Smith', 'e-mailed', 'the', 'file', 'at', '5:00', 'p.m.', 'to', 'john', '@', 'example.com']

Замечание: NLTK лучше разделяет пунктуацию


In [13]:
print("\n" + "=" * 60)
print("ЗАДАЧА 3: Создание собственного токенизатора")
print("=" * 60)

def advanced_tokenize(text):
    """
    Продвинутая токенизация с обработкой:
    - Email адресов
    - URL
    - Хэштегов
    - Чисел и денежных сумм
    """
    # Сохраняем специальные токены
    tokens = []
    
    # Паттерны
    patterns = [
        (r'https?://\S+', 'URL'),
        (r'\S+@\S+', 'EMAIL'),
        (r'#\w+', 'HASHTAG'),
        (r'\$?\d+[.,]\d+', 'NUMBER'),
        (r'\b\w+\b', 'WORD'),
        (r'[.,:;!?]', 'PUNCT'),
    ]
    
    from itertools import chain
    combined_pattern = '|'.join(f'(?P<{name}>{pat})' for pat, name in patterns)
    
    for match in re.finditer(combined_pattern, text):
        token = match.group()
        token_type = match.lastgroup
        tokens.append((token, token_type))
    
    return tokens

test = "Check my email at john@example.com or visit https://example.com. Price is $19.99 #NLP"
result = advanced_tokenize(test)

print(f"\nТекст: {test}")
print(f"\nТокены с типами:")
for token, token_type in result:
    print(f"  {token:20} → {token_type}")


ЗАДАЧА 3: Создание собственного токенизатора

Текст: Check my email at john@example.com or visit https://example.com. Price is $19.99 #NLP

Токены с типами:
  Check                → WORD
  my                   → WORD
  email                → WORD
  at                   → WORD
  john@example.com     → EMAIL
  or                   → WORD
  visit                → WORD
  https://example.com. → URL
  Price                → WORD
  is                   → WORD
  $19.99               → NUMBER
  #NLP                 → HASHTAG


In [14]:
print("\n" + "=" * 60)
print("ЗАДАЧА 4: Влияние стоп-слов на классификацию")
print("=" * 60)

sample_text = "Это очень интересный и важный документ о машинном обучении"

tokens = word_tokenize(sample_text.lower(), language='russian')
stop_words_ru = stopwords.words('russian')

print(f"\nОригинальные токены: {[t for t in tokens if t.isalpha()]}")

# С удалением стоп-слов
filtered = [t for t in tokens if t.isalpha() and t not in stop_words_ru]
print(f"Без стоп-слов: {filtered}")

removed = [t for t in tokens if t.isalpha() and t in stop_words_ru]
print(f"\nУдалены стоп-слова: {removed}")
print(f"\nОригинальных слов: {sum(1 for t in tokens if t.isalpha())}")
print(f"Значимых слов: {len(filtered)}")
print(f"Удалено: {len(removed)} ({len(removed)/sum(1 for t in tokens if t.isalpha()):.1%})")


ЗАДАЧА 4: Влияние стоп-слов на классификацию

Оригинальные токены: ['это', 'очень', 'интересный', 'и', 'важный', 'документ', 'о', 'машинном', 'обучении']
Без стоп-слов: ['это', 'очень', 'интересный', 'важный', 'документ', 'машинном', 'обучении']

Удалены стоп-слова: ['и', 'о']

Оригинальных слов: 9
Значимых слов: 7
Удалено: 2 (22.2%)
